# Import Necessary Libraries

In [1]:
import numpy as np
import os
import mne
import pywt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.interpolate import interp1d
from scipy.stats import skew
from scipy.signal import welch
from scipy.stats import entropy

# Data and Labels

In [2]:
# Load the saved features from the .npz file
data = np.load('combined_eceo_data.npz') 

# Access the concatenated features from the .npz file
X = data['X']

# Close the loaded file
data.close()

#print the merged array
X

array([[[-2.90148955e-06,  1.12909709e-06,  2.63543120e-06, ...,
         -2.89727495e-06, -6.80166352e-07,  1.37569339e-06],
        [-1.20626610e-05, -1.10767687e-05, -1.06193111e-05, ...,
         -5.28302598e-06, -3.08162134e-06, -1.08686640e-06],
        [-3.79762923e-07, -2.07405219e-06, -4.63428751e-06, ...,
         -1.40524844e-06, -2.77773978e-07,  5.66622077e-07],
        ...,
        [ 1.29488217e-06,  1.93576102e-06,  1.93742471e-06, ...,
         -5.94415587e-06, -6.17930937e-06, -4.59392322e-06],
        [ 6.58896488e-06,  7.07184518e-06,  7.21316883e-06, ...,
          1.58482089e-06,  1.10701546e-06,  1.01823366e-06],
        [ 2.77446316e-06, -3.67728944e-06, -5.35505842e-06, ...,
          1.31410061e-05,  9.10587005e-06,  1.86895170e-06]],

       [[ 1.65448073e-05,  1.31738316e-05,  9.04923854e-06, ...,
         -4.42389546e-06, -3.17349259e-06, -1.59048273e-06],
        [ 4.49196432e-06,  3.74644878e-06,  1.34007439e-06, ...,
         -9.20414914e-06, -8.74245961e

In [3]:
data = np.load('combined_eceo_labels.npz')

Y = data['Y']

data.close() 

Y

array([0, 0, 0, ..., 1, 1, 1])

In [4]:
data = np.load('combined_eceo_groups.npz')

group = data['group']

data.close() 

group

array([0, 0, 0, ..., 1, 1, 1])

# Functions of Feature Extraction

## Time Domain Features

In [5]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import find_peaks

def zero_crossing_rate(signal):
    zero_crossings = np.where(np.diff(np.sign(signal)))[0]
    return len(zero_crossings) / len(signal)

def hjorth_parameters(signal):
    diff_input = np.diff(signal)
    diff_diff_input = np.diff(diff_input)

    activity = np.var(signal)
    mobility = np.sqrt(np.var(diff_input)/activity)
    complexity = np.sqrt(np.var(diff_diff_input)/np.var(diff_input)) / mobility

    return activity, mobility, complexity

def extract_time_domain_features(epochs):
    features = []

    for epoch in epochs:
        epoch_features = []

        for channel_data in epoch:
            # Flatten the channel data
            flattened_data = channel_data.flatten()

            # Basic Time-Domain Features
            mean_val = np.mean(flattened_data)
            median_val = np.median(flattened_data)
            var_val = np.var(flattened_data)
            std_dev = np.std(flattened_data)
            skewness = skew(flattened_data)
            kurt = kurtosis(flattened_data)
            zcr = zero_crossing_rate(flattened_data)
            peak_amp = np.ptp(flattened_data)

            # Hjorth Parameters
            activity, mobility, complexity = hjorth_parameters(flattened_data)

            # Additional Features
            num_waves = len(find_peaks(flattened_data)[0])
            wave_duration = len(flattened_data) / num_waves if num_waves > 0 else 0

            channel_features = [
                mean_val, median_val, var_val, std_dev, skewness, kurt, zcr, num_waves,
                wave_duration, peak_amp, activity, mobility, complexity
            ]
            epoch_features.append(channel_features)

        features.append(epoch_features)

    return np.array(features)

## Frequency Domain Features

In [6]:
import numpy as np
from scipy.stats import skew, kurtosis
from scipy.signal import welch



def get_wavelet_coeffs(channel_data, wavelet='db4', level=3):
    coeffs = pywt.wavedec(channel_data, wavelet, level=level)
    return coeffs


def extract_frequency_domain_features(epochs, sfreq,wavelet='db4', bands={'delta': (1, 3), 'theta': (4, 7), 'alpha': (8, 12), 'beta': (13, 30), 'gamma': (31, 60), 'sigma': (11, 16)}):
    features = []

    for epoch in epochs:
        epoch_features = []

        for channel_data in epoch:
            # Compute the Power Spectral Density (PSD)
            freqs, psd = welch(channel_data, sfreq, nperseg=180)

            # Frequency domain features
            mean_val = np.mean(psd)
            median_val = np.median(psd)
            var_val = np.var(psd)
            std_dev = np.std(psd)
            skewness = skew(psd)
            kurt = kurtosis(psd)

            # Compute wavelet coefficients
            wave_coeffs = get_wavelet_coeffs(channel_data, wavelet, level=3)
            wave_coeffs_mean = np.mean(wave_coeffs[0])

            # Band Power Features
            band_powers = {}
            for band, freq_range in bands.items():
                freq_mask = (freqs >= freq_range[0]) & (freqs <= freq_range[1])
                band_power = np.sum(psd[freq_mask])
                band_powers[band] = band_power

            # Band Power Ratios
            theta_alpha_ratio = band_powers['theta'] / band_powers['alpha']
            beta_alpha_ratio = band_powers['beta'] / band_powers['alpha']
            theta_alpha_beta_ratio = (band_powers['theta'] + band_powers['alpha']) / band_powers['beta']
            # Additional Band Power Ratios
            theta_beta_ratio = band_powers['theta'] / band_powers['beta']
            theta_alpha_beta_alpha_ratio = (band_powers['theta'] + band_powers['alpha']) / (band_powers['alpha'] + band_powers['beta'])
            gamma_delta_ratio = band_powers['gamma'] / band_powers['delta']
            gamma_beta_delta_alpha_ratio = (band_powers['gamma'] + band_powers['beta']) / (band_powers['delta'] + band_powers['alpha'])


            channel_features = [
                mean_val, median_val, var_val, std_dev, skewness, kurt,
                band_powers['delta'], band_powers['theta'], band_powers['alpha'],
                band_powers['beta'], band_powers['gamma'], band_powers['sigma'],
                theta_alpha_ratio, beta_alpha_ratio, theta_alpha_beta_ratio,theta_beta_ratio,
                theta_alpha_beta_alpha_ratio, gamma_delta_ratio, gamma_beta_delta_alpha_ratio,
                wave_coeffs_mean
            ]
            epoch_features.append(channel_features)

        features.append(epoch_features)

    return np.array(features)

## Descrete Wavelet Transform (DWT) Features

In [7]:
import pywt

def extract_dwt_features(epochs, wavelet='db4', level=4):
   
    n_epochs, n_channels, _ = epochs.shape
    dwt_features = []

    for epoch in epochs:
        epoch_features = []
        for channel in epoch:
            # Perform DWT
            coeffs = pywt.wavedec(channel, wavelet, level=level)
            
            # Concatenate DWT coefficients
            concatenated_coeffs = np.concatenate(coeffs, axis=0)
            
            epoch_features.append(concatenated_coeffs)
        
        dwt_features.append(epoch_features)

    return np.array(dwt_features)

## Detrended Flactuation Analysis (DFA)

In [5]:
def extract_dfa_features(epochs, min_window_size, max_window_size):
    
    n_epochs, n_channels, _ = epochs.shape
    dfa_features = []

    for epoch in epochs:
        epoch_features = []
        for channel in epoch:
            channel_features = []
            for w in range(min_window_size, max_window_size + 1):
                # Compute DFA
                dfa = compute_dfa(channel, w)
                channel_features.append(dfa)
            epoch_features.append(channel_features)
        dfa_features.append(epoch_features)

    return np.array(dfa_features)

def compute_dfa(data, window_size):
    """Compute Detrended Fluctuation Analysis (DFA) for a single channel."""
    # Compute cumulative sum of the data
    cumsum = np.cumsum(data - np.mean(data))

    # Divide the signal into non-overlapping windows of size window_size
    windows = np.array_split(cumsum, len(data) // window_size)

    # Compute the local trend within each window
    trends = [np.polyfit(np.arange(len(window)), window, 1)[0] for window in windows]

    # Compute the root mean square of the integrated fluctuation within each window
    flucts = np.array([np.sqrt(np.mean((window - trend) ** 2)) for window, trend in zip(windows, trends)])

    # Compute the DFA value as the slope of the log-log plot of window size vs. fluctuation
    dfa = np.polyfit(np.log(np.arange(len(flucts)) + 1), np.log(flucts), 1)[0]

    return dfa

## Higuchi's Fractual Dimension (HFD)

In [3]:
def extract_hfd_features(epochs, kmax):
   
    n_epochs, n_channels, _ = epochs.shape
    hfd_features = []

    for epoch in epochs:
        epoch_features = []
        for channel in epoch:
            channel_features = []
            for k in range(2, kmax+1):
                # Compute Higuchi's Fractal Dimension
                hfd = compute_hfd(channel, k)
                channel_features.append(hfd)
            epoch_features.append(channel_features)
        hfd_features.append(epoch_features)

    return np.array(hfd_features)


def compute_hfd(data, kmax):
    """Compute Higuchi's Fractal Dimension for a single channel."""
    n = len(data)
    lk = np.zeros(kmax)

    for k in range(2, kmax + 1):
        lmk = 0.0
        for m in range(1, k + 1):
            lk_m_sum = 0.0
            for i in range(1, int(np.floor((n - m) / k)) + 1):
                lk_m_sum += abs(data[m + i * k - 1] - data[m + (i - 1) * k - 1])
            lk_m_sum *= (n - 1) / (k * np.floor((n - m) / k))
            lmk += lk_m_sum / ((n - 1) / k)
        lmk /= k
        lk[k - 1] = lmk

    return np.log(lk) / np.log(np.arange(2, kmax + 2))

## Lemple - Ziv Complexity (LZC)

In [3]:
def extract_lzc_features(epochs):
    n_epochs, n_channels, _ = epochs.shape
    lzc_features = []

    for epoch in epochs:
        epoch_features = []
        for channel in epoch:
            # Compute LZC for each channel
            lzc = compute_lzc(channel)
            epoch_features.append(lzc)
        lzc_features.append(epoch_features)

    return np.array(lzc_features)


def compute_lzc(data):
    """Compute Lempel-Ziv Complexity (LZC) for a single channel."""
    n = len(data)
    u = [data[0]]
    complexity = 1

    for i in range(1, n):
        w = data[i]
        if w not in u:
            u.append(w)
            complexity += 1

    return complexity / n

## Power Spectral Density (PSD)

In [4]:
def extract_psd_features(epochs, sfreq):
    psd_features = []
    
    for epoch in epochs:
        epoch_features = []
        
        for channel_data in epoch:
            # Compute Power Spectral Density (PSD)
            freqs, psd = welch(channel_data, fs=sfreq)
            
            # Integrate PSD over frequency bands
            delta_power = np.trapz(psd[(freqs >= 0.5) & (freqs <= 4)], x=freqs[(freqs >= 0.5) & (freqs <= 4)])
            theta_power = np.trapz(psd[(freqs > 4) & (freqs <= 8)], x=freqs[(freqs > 4) & (freqs <= 8)])
            alpha_power = np.trapz(psd[(freqs > 8) & (freqs <= 13)], x=freqs[(freqs > 8) & (freqs <= 13)])
            beta_power = np.trapz(psd[(freqs > 13) & (freqs <= 30)], x=freqs[(freqs > 13) & (freqs <= 30)])
            gamma_power = np.trapz(psd[(freqs > 30) & (freqs <= 100)], x=freqs[(freqs > 30) & (freqs <= 100)])
            
            # Total power
            total_power = np.trapz(psd, x=freqs)
            
            epoch_features.append([delta_power, theta_power, alpha_power, beta_power, gamma_power, total_power])
        
        psd_features.append(epoch_features)
    
    return np.array(psd_features)

## Spectral Entropy

In [5]:
def extract_entropy_features(epochs):
    entropy_features = []
    
    for epoch in epochs:
        epoch_features = []
        
        for channel_data in epoch:
            # Compute power spectral density
            freqs, psd = welch(channel_data)
            
            # Normalize PSD
            psd /= np.sum(psd)
            
            # Compute Shannon entropy
            entropy_val = -np.sum(psd * np.log2(psd))
            epoch_features.append(entropy_val)
        
        entropy_features.append(epoch_features)
    
    return np.array(entropy_features)

## Statistical Features

### Mean

In [6]:
def extract_mean_features(epochs):
    mean_features = []
    
    for epoch in epochs:
        epoch_features = []
        
        for channel_data in epoch:
            # Compute mean value
            mean_val = np.mean(channel_data)
            epoch_features.append(mean_val)
        
        mean_features.append(epoch_features)
    
    return np.array(mean_features)


### Variance

In [7]:
def extract_variance_features(epochs):
    variance_features = []
    
    for epoch in epochs:
        epoch_features = []
        
        for channel_data in epoch:
            # Compute variance
            var_val = np.var(channel_data)
            epoch_features.append(var_val)
        
        variance_features.append(epoch_features)
    
    return np.array(variance_features)

### Skewness

In [8]:
def extract_skewness_features(epochs):
    skewness_features = []
    
    for epoch in epochs:
        epoch_features = []
        
        for channel_data in epoch:
            # Compute skewness
            skew_val = skew(channel_data)
            epoch_features.append(skew_val)
        
        skewness_features.append(epoch_features)
    
    return np.array(skewness_features)

## Continuous Wavelet Transform (CWT)

In [ ]:
import pywt

def extract_cwt_features(epochs, wavelet='morl', scales=np.arange(1, 128)):
    n_epochs, n_channels, _ = epochs.shape
    cwt_features = []

    for epoch in epochs:
        epoch_cwt_features = []
        
        for channel in epoch:
            # Perform CWT
            cwt_coeffs, _ = pywt.cwt(channel, scales, wavelet)
            epoch_cwt_features.append(cwt_coeffs)

        cwt_features.append(epoch_cwt_features)

    return np.array(cwt_features)

# Extract and Save Features

## Time Domain Features

In [8]:
X_time = extract_time_domain_features(X)

In [9]:
# Define the file path to save the features
save_path = 'combined_eceo_time.npz'

# # Save the features
np.savez(save_path, X_time=X_time)

print("Features saved successfully.")

Features saved successfully.


## Frequency Domain Features

In [10]:
sfreq = 256  # Replace with the sampling frequency of your data
# epochs_data = [epoch.get_data() for epoch in epochs] 
X_frequency = extract_frequency_domain_features(X, sfreq)

In [11]:
# Define the file path to save the features
save_path = 'combined_eceo_freq.npz'

# Save the features
np.savez(save_path, X_frequency=X_frequency)

print("Features saved successfully.")

Features saved successfully.


## DWT Features

In [13]:
# Extract DWT features
X_dwt = extract_dwt_features(X)

In [14]:
# Define the file path to save the features
save_path = 'combined_eceo_dwt.npz'

# Save the features
np.savez(save_path, X_dwt=X_dwt)

print("Features saved successfully.")

Features saved successfully.


## DFA

In [4]:
# Define the minimum and maximum window sizes
min_window_size = 2  # Since each epoch is 1 second long
max_window_size = int(np.log2(X.get_data().shape[2]))

# Extract DFA features
X_dfa = extract_dfa_features(X.get_data(), min_window_size, max_window_size)

C:\Users\Admin\AppData\Local\Temp\ipykernel_13724\2949197905.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  max_window_size = int(np.log2(X.get_data().shape[2]))
C:\Users\Admin\AppData\Local\Temp\ipykernel_13724\2949197905.py:6: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_dfa = extract_dfa_features(X.get_data(), min_window_size, max_window_size)


In [5]:
# Define the file path to save the features
save_path = 'combined_dfa.npz'

# Save the features
np.savez(save_path, X_dfa=X_dfa)

print("Features saved successfully.")

Features saved successfully.


## HFD

In [ ]:
sfreq = 256
kmax = sfreq//2
# Extract Higuchi's Fractal Dimension features
X_hfd = extract_hfd_features(X.get_data(), kmax)

C:\Users\Admin\AppData\Local\Temp\ipykernel_17972\1036302989.py:4: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_hfd = extract_hfd_features(X.get_data(), kmax)
C:\Users\Admin\AppData\Local\Temp\ipykernel_17972\1668856596.py:36: RuntimeWarning: divide by zero encountered in log
  return np.log(lk) / np.log(np.arange(2, kmax + 2))
C:\Users\Admin\AppData\Local\Temp\ipykernel_17972\1668856596.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  lk_m_sum *= (n - 1) / (k * np.floor((n - m) / k))
C:\Users\Admin\AppData\Local\Temp\ipykernel_17972\1668856596.py:31: RuntimeWarning: invalid value encountered in scalar multiply
  lk_m_sum *= (n - 1) / (k * np.floor((n - m) / k))


In [ ]:
# Define the file path to save the features
save_path = 'combined_hfd.npz'

# Save the features
np.savez(save_path, X_hfd=X_hfd)

print("Features saved successfully.")

## LZC

In [9]:
# Extract LZC features
X_lzc = extract_lzc_features(X.get_data())

C:\Users\Admin\AppData\Local\Temp\ipykernel_22000\1568084668.py:2: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_lzc = extract_lzc_features(X.get_data())


In [10]:
# Define the file path to save the features
save_path = 'combined_lzc.npz'

# Save the features
np.savez(save_path, X_lzc=X_lzc)

print("Features saved successfully.")

Features saved successfully.


## PSD

In [11]:
# Define the sampling frequency
sfreq = 256

# Extract spectral features
X_psd = extract_psd_features(X.get_data(), sfreq)

C:\Users\Admin\AppData\Local\Temp\ipykernel_22000\3757737867.py:5: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_psd = extract_psd_features(X.get_data(), sfreq)
C:\Users\Admin\anaconda3\envs\mne\Lib\site-packages\scipy\signal\_spectral_py.py:589: UserWarning: nperseg = 256 is greater than input length  = 180, using nperseg = 180
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


In [12]:
# Define the file path to save the features
save_path = 'combined_psd.npz'

# Save the features
np.savez(save_path, X_psd=X_psd)

print("Features saved successfully.")

Features saved successfully.


## Spectral Entropy

In [13]:
X_entropy = extract_entropy_features(X.get_data())

C:\Users\Admin\AppData\Local\Temp\ipykernel_22000\1377404312.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_entropy = extract_entropy_features(X.get_data())


In [14]:
# Define the file path to save the features
save_path = 'combined_entropy.npz'

# Save the features
np.savez(save_path, X_entropy=X_entropy)

print("Features saved successfully.")

Features saved successfully.


## Statistical

### Mean

In [15]:
X_mean = extract_mean_features(X.get_data())

C:\Users\Admin\AppData\Local\Temp\ipykernel_22000\730594860.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_mean = extract_mean_features(X.get_data())


In [16]:
# Define the file path to save the features
save_path = 'combined_mean.npz'

# Save the features
np.savez(save_path, X_mean=X_mean)

print("Features saved successfully.")

Features saved successfully.


### Variance

In [17]:
X_variance = extract_variance_features(X.get_data())

C:\Users\Admin\AppData\Local\Temp\ipykernel_22000\1504960069.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_variance = extract_variance_features(X.get_data())


In [18]:
# Define the file path to save the features
save_path = 'combined_variance.npz'

# Save the features
np.savez(save_path, X_variance=X_variance)

print("Features saved successfully.")

Features saved successfully.


### Skewness

In [19]:
X_skewness = extract_skewness_features(X.get_data())

C:\Users\Admin\AppData\Local\Temp\ipykernel_22000\4238270862.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  X_skewness = extract_skewness_features(X.get_data())


In [20]:
# Define the file path to save the features
save_path = 'combined_skewness.npz'

# Save the features
np.savez(save_path, X_skewness=X_skewness)

print("Features saved successfully.")

Features saved successfully.


## CWT Features

In [ ]:
# # Extract CWT features
# X_cwt = extract_cwt_features(X.get_data())

In [ ]:
# # Define the file path to save the features
# save_path = 'combined_cwt.npz'

# # Save the features
# np.savez(save_path, X_cwt=X_cwt)

# print("Features saved successfully.")

# Complete